In [1]:
import numpy as np
from copy import copy
import tensorflow as tf
from importlib import reload
from random import randint
import matplotlib.pyplot as plt

import environment as env
import agent
import agents.random_agent as ra
import agents.q_no_hidden as qnh
import agents.human as human

In [ ]:
reload(env)
reload(agent)
reload(ra)
reload(qnh)
reload(human)

In [2]:
a1 = qnh.QNoHidden(3)
a2 = ra.RandomAgent()
h = human.Human()
e = env.Env(3)
e.train(a1, h, 2, True)

Turn #0

O O O 
O O O 
O O X 

Turn #1
Next Piece: 1 1

X O O 
O O O 
O O X 

Turn #2

X O O 
O O O 
O O X 

Player attempted illegal move
Player 2 Wins!
Turn #0

X O O 
O O O 
O O O 

Turn #1
Next Piece: 2 2

X O O 
O X O 
O O O 

Turn #2

X O O 
X X O 
O O O 

Turn #3
Next Piece: 3 3

X O O 
X X O 
O O X 

Player 1 Wins!


In [3]:
e.reset()
print(np.reshape(a1.get_Q(e.observe()), [3,3]))

[[  5.81911325e-01  -8.30414832e-01  -1.59635007e+00]
 [  1.14921883e-01  -1.00835077e-01  -8.62590492e-01]
 [ -1.13725066e+00  -1.01383519e+00  -1.99260910e+02]]
